In [45]:
import spotipy
from spotipy import util
from pprint import pprint
import random
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np

In [46]:
username = 'ernyjyqoecerasugt8dpmhbmo'
scope = 'playlist-modify-public'
my_client_id = '83c842b2f4d447b6b2a19df91847487d'
my_client_secret = '8bd6101237114d9eb851e2d3183affce'

spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

In [47]:
csv_df = pd.read_csv("Resources/Hot_100BBCharts_time.csv")
csv_df.head()

,Date,Title,Artist,Peak_Position,Last_Position,Weeks_on_Chart,Rank_in_Chart,New_on_Chart
0,1958-08-06,Poor Little Fool,Ricky Nelson,1,1,2,1,False
1,1958-08-06,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,2,54,2,2,False
2,1958-08-06,Patricia,Perez Prado And His Orchestra,2,2,2,3,False
3,1958-08-06,Splish Splash,Bobby Darin,3,3,2,4,False
4,1958-08-06,When,Kalin Twins,5,5,2,5,False


In [48]:
sp = spotipy.Spotify(auth=spotify_token)

In [70]:
index = np.arange(0,100,1)

dfcolumns = ["Test Song","Song & Artist","Song Name Match Score","Artist Name Match Score","Overall Match Score"]

song_results_df = pd.DataFrame(index=index,columns=dfcolumns)

test_song_name = []
song_name = []
match_score_song = []
match_score_artist = []
match_score = []

for i in range(100):
    test_song = csv_df.iloc[i,1]+", "+csv_df.iloc[i,2]

    results = sp.search(q=test_song, type='track', limit=10)

    search_song_name = []
    search_match_score_song = []
    search_match_score_artist = []
    search_match_score = []

    try:
    
        for x in range(len(results["tracks"]["items"])):
            song = results["tracks"]["items"][x]["name"]
            artist = results["tracks"]["items"][x]["artists"][0]["name"]
            search_song_name.append(song+", "+artist)
            search_match_score.append(fuzz.ratio(test_song,song+","+artist))
            search_match_score_song.append(fuzz.ratio(csv_df.iloc[i,1],song))
            search_match_score_artist.append(fuzz.ratio(csv_df.iloc[i,2],artist))

        max_loc = 0
        max_val = 0
        for y in range(len(results["tracks"]["items"])):
            if max_val < search_match_score[y]:
                max_loc = y
                max_val = search_match_score[y]
                
        test_song_name.append(test_song)
        song_name.append(search_song_name[max_loc])
        match_score_song.append(search_match_score_song[max_loc])
        match_score_artist.append(search_match_score_artist[max_loc])
        match_score.append(search_match_score[max_loc])
    
    except:
        song_name.append("")
        match_score_song.append(0)
        match_score_artist.append(0)
        match_score.append(0)


    
        
    #pprint(song_name)
    #pprint(match_score)

    #song_id1 = results["tracks"]["items"][1]["id"]
    #song_id2 = results["tracks"]["items"][2]["id"]
    #feats1 = sp.audio_features(song_id1)
    #feats2 = sp.audio_features(song_id2)

    #print(feats1)
    #print(feats2)

    #song_match = pd.DataFrame([song_name,match_score_song,match_score_artist,match_score]).transpose()
    #song_match
    
song_results_df["Test Song"] = test_song_name
song_results_df["Song & Artist"] = song_name
song_results_df["Song Name Match Score"] = match_score_song
song_results_df["Artist Name Match Score"] = match_score_artist
song_results_df["Overall Match Score"] = match_score
song_results_df

,Test Song,Song & Artist,Song Name Match Score,Artist Name Match Score,Overall Match Score
0,"Poor Little Fool, Ricky Nelson","Poor Little Fool, Ricky Nelson",100,100,98
1,"Nel Blu Dipinto Di Blu (Volaré), Domenico Modugno","Nel Blu Dipinto Di Blu (Volare), Domenico Modugno",97,100,97
2,"Patricia, Perez Prado And His Orchestra",,0,0,0
3,"Splish Splash, Bobby Darin","Splish Splash, Bobby Darin",100,100,98
4,"When, Kalin Twins","When - Live, Kalin Twins",53,100,80
5,"My True Love, Jack Scott","My True Love, Jack Scott",100,100,98
6,"Hard Headed Woman, Elvis Presley With The Jord...",,0,0,0
7,"Rebel-'rouser, Duane Eddy His Twangy Guitar An...",,0,0,0
8,"Just A Dream, Jimmy Clanton And His Rockets","Karaoke - Just A Dream, Karaoke - Jimmy Clanto...",71,79,78
9,"Willie And The Hand Jive, The Johnny Otis Show","Willie And The Hand Jive - Live, Johnny Otis",87,71,83


In [78]:
song_results_df["Overall Match Score"].value_counts()
song_results_df.loc[song_results_df["Overall Match Score"] <= 70]

,Test Song,Song & Artist,Song Name Match Score,Artist Name Match Score,Overall Match Score
2,"Patricia, Perez Prado And His Orchestra",,0,0,0
6,"Hard Headed Woman, Elvis Presley With The Jord...",,0,0,0
7,"Rebel-'rouser, Duane Eddy His Twangy Guitar An...",,0,0,0
18,Left Right Out Of Your Heart (Hi Lee Hi Lo Hi ...,,0,0,0
24,"Somebody Touched Me, Buddy Knox with the Rhyth...",,0,0,0
29,"Moon Talk, Perry Como",,0,0,0
32,"Rock-in Robin, Bobby Day","Karaoke - Rock-In Robin, Karaoke - Bobby Day",67,64,69
36,"Volare (Nel Blu Dipinto Di Blu), Dean Martin",Volare (Nel Blu Dipinto Di Blu) [Karaoke Versi...,78,27,70
53,"Don't Ask Me Why, Elvis Presley With The Jorda...",,0,0,0
56,"That's How Much I Love You, Pat Boone",,0,0,0


In [3]:
fuzz.ratio("Song Name (feat Artist), Artist Name", "Song Name, Artist Name")

76

In [5]:
fuzz.ratio("Song Name - Live, Artist Name", "Song Name, Artist Name")

86

In [7]:
fuzz.ratio("Heart Shaped Box, Nirvana","Smells Like Teen Spirit, Nirvana")

46